In [3]:
!conda install pytorch::pytorch torchvision torchaudio -c pytorch -y


Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniforge/base/envs/mura

  added / updated specs:
    - pytorch::pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pillow-10.0.1              |  py311h8dc27b9_2        43.5 MB  conda-forge
    pytorch-2.1.0              |         py3.11_0        53.6 MB  pytorch
    torchaudio-2.1.0           |        py311_cpu         4.8 MB  pytorch
    torchvision-0.16.0         |        py311_cpu         7.0 MB  pytorch
    ------------------------------------------------------------
                                           Total:       108.8 MB

The following NEW packages will be INSTALLED:

  aom                conda-forge/osx-arm64::aom-3.6.1-hb765f3a_0 
  cairo              conda-forge/osx-arm64::cairo-1.16.0-hc5b65c1_1017 
  dav1d              conda-forge/os

In [20]:
!pip install -q transformers datasets pytorch-lightning


In [4]:
!pip install pandas


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/af/7b/d170f9c8306c7673f57ca4f442e326d36e20299725edc5d0af36a3e3b041/pandas-2.1.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 7.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [41]:
import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder


In [42]:
import shutil


In [38]:
if (os.path.exists("Mura")):
    # !rm -rf Mura
    shutil.rmtree("Mura")
else:
    os.mkdir("Mura")
    os.mkdir("Mura/train")
    os.mkdir("Mura/valid")

train = pd.read_csv("./Train_set.csv")
valid = pd.read_csv("./Valid_set.csv")

for i in range(len(train)):
    train_path = train.iloc[i, 0]
    file_name = train_path.split("/")[-1]
    if (train.iloc[i, 1] == 1):

        if (os.path.exists("Mura/train/positive")):
            pass
        else:
            os.makedirs("Mura/train/positive")
        shutil.copy(train_path, f"Mura/train/positive/{i}-{file_name}")
    else:
        if (os.path.exists("Mura/train/negative")):
            pass
        else:
            os.mkdir("Mura/train/negative")
        shutil.copy(train_path, f"Mura/train/negative/{i}-{file_name}")
for i in range(len(valid)):
    valid_path = valid.iloc[i, 0]
    file_name = valid_path.split("/")[-1]

    if (valid.iloc[i, 1] == 1):
        if (os.path.exists("Mura/valid/positive")):
            pass
        else:
            os.mkdir("Mura/valid/positive")
        shutil.copy(valid_path, f"Mura/valid/positive/{i}-{file_name}")
    else:
        if (os.path.exists("Mura/valid/negative")):
            pass
        else:
            os.mkdir("Mura/valid/negative")
        shutil.copy(valid_path, f"Mura/valid/negative/{i}-{file_name}")


In [43]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
image_mean = processor.image_mean
image_std = processor.image_std
size = processor.size["height"]


In [44]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples


In [60]:
# Create a dataset from the list of tuples
# dataset = ImageFolder("MURA-v1.1/train/", transform=)
train_ds = ImageFolder("Mura/train/", transform=train_transforms)


# Create a data loader from the dataset
dataloader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)


In [51]:
train_ds


Dataset ImageFolder
    Number of datapoints: 36808
    Root location: Mura/train/
    StandardTransform
Transform: <function train_transforms at 0x289f23060>

In [61]:
val_ds = ImageFolder("Mura/valid/", transform=val_transforms)
dataloader_val = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)


In [53]:
val_ds, dataloader_val


(Dataset ImageFolder
     Number of datapoints: 3197
     Root location: Mura/valid/
     StandardTransform
 Transform: <function val_transforms at 0x289f21d00>,
 <torch.utils.data.dataloader.DataLoader at 0x17e488b90>)

In [55]:
from torch.utils.data import DataLoader

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_batch_size = 2
eval_batch_size = 2

train_dataloader = DataLoader(train_ds, shuffle=True, collate_fn=collate_fn, batch_size=train_batch_size)
val_dataloader = DataLoader(val_ds, collate_fn=collate_fn, batch_size=eval_batch_size)
# test_dataloader = DataLoader(test_ds, collate_fn=collate_fn, batch_size=eval_batch_size)


In [66]:
next(iter(val_ds))


TypeError: 'Image' object is not subscriptable